### Создание DataSet / Парсинг по разделам journal.ksk.expert

In [13]:
import requests
from bs4 import BeautifulSoup
import json
from pprint import pprint
import re

session = requests.Session()
#Ссылка на авторизацию POST запрос
link_auth = 'https://journal.ksk.expert/login/?login=yes'
#Ссылка на раздел который парсим
URL = 'https://journal.ksk.expert/articles/hr-keysy/'

HEADERS = {
    'user-agent': 'Mozilla/5.0 (Linux; Android 6.0; Nexus 5 Build/MRA58N) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/101.0.4951.64 Mobile Safari/537.36',
    'accept': '*/*'
}

data_auth = {
    'backurl': '/login/',
    'AUTH_FORM': 'Y',
    'TYPE': 'AUTH',
    'USER_LOGIN': 'maltseivan',
    'USER_PASSWORD': '*********',
    'Login': 'Войти'
}

responce = session.post(link_auth, data=data_auth, headers=HEADERS)
# Ответ сервера на POST запрос с авторизацией: print(responce)

#data = Храняться полученные статьи
data_list = {}

def get_html(url, params=None):
    r = session.get(url, headers=HEADERS, params=params)
    return r
def get_content(html):
    #Парсер ссылок раздела
    soup = BeautifulSoup(html, 'html.parser')
    link = soup.find_all('a', class_='link-article')
    cat = soup.find('h1', class_='ch-title').text
    #Раздел в котором парсим pprint(cat)
    
    for elem in link:
        link_href_attr = elem.get("href")
        urlink = 'https://journal.ksk.expert' + link_href_attr
        
        #Парсер статьи по ссылкам
        def get_html_article(urlink, params=None):
            r_text = session.get(urlink, headers=HEADERS, params=params)
            return r_text

        def get_content_article(html):
            soup = BeautifulSoup(html, 'html.parser')
            chapter_article = soup.find('h1', class_='ch-title').text
            content_article = soup.find_all('div', class_='contents mb-60')
            #Название статей в разделе pprint(chapter_article)

            for elem in content_article:
                index_text = elem.text
                #Контентая часть статьи pprint(index_text[0])
                data_list[chapter_article] = [index_text]
                
                with open(cat + '.json', 'w', encoding='utf-8') as file:
                    json.dump(data_list, file, indent=3, ensure_ascii=False)

        def parse_article():
            result = get_html_article(urlink)
            if result.status_code == 200:
                get_content_article(result.text)
            else:
                print('Не могу получить страницу!')
         
        parse_article();
        

def parse():
    result = get_html(URL)
    if result.status_code == 200:
        get_content(result.text)
    else:
        print('Не могу получить страницу!')


parse();


In [5]:
import json
from pprint import pprint

data = {}

with open("HR-кейсы.json", "r", encoding='utf-8') as file:
    data['HR-кейсы'] = json.load(file)
    
with open("HR-тесты.json", "r", encoding='utf-8') as file:
    data['HR-тесты'] = json.load(file) 
    
with open("HR-юмор.json", "r", encoding='utf-8') as file:
    data['HR-юмор'] = json.load(file) 
    
with open("Автоматизация HR.json", "r", encoding='utf-8') as file:
    data['Автоматизация HR'] = json.load(file) 
    
with open("Адаптация.json", "r", encoding='utf-8') as file:
    data['Адаптация'] = json.load(file)
    
with open("Бизнес.json", "r", encoding='utf-8') as file:
    data['Бизнес'] = json.load(file)

with open("Корпоративная культура.json", "r", encoding='utf-8') as file:
    data['Корпоративная культура'] = json.load(file)
    
with open("Мотивация.json", "r", encoding='utf-8') as file:
    data['Мотивация'] = json.load(file)  
    
with open("Обучение персонала.json", "r", encoding='utf-8') as file:
    data['Обучение персонала'] = json.load(file) 
    
with open("Рекрутмент.json", "r", encoding='utf-8') as file:
    data['Рекрутмент'] = json.load(file) 

with open("Трудовое законодательство.json", "r", encoding='utf-8') as file:
    data['Трудовое законодательство'] = json.load(file) 

with open("Управление персоналом.json", "r", encoding='utf-8') as file:
    data['Управление персоналом'] = json.load(file) 
    
with open('data.json', 'w', encoding='utf-8') as file:
    json.dump(data, file, indent=3, ensure_ascii=False)
    

### Создание DataFrame

In [6]:
from tqdm import tqdm
import pandas as pd

category = []
name_article = []
article = []

for elem in data:
    for value in data[elem]:
        for text in data[elem][value]:
            category.append(elem)
            name_article.append(value)
            article.append(text)
df_data = {
    'Категория': category,
    'Название статьи': name_article,
    'Статья': article
}

df = pd.DataFrame(df_data)
df

,Категория,Название статьи,Статья
0,HR-кейсы,Как повысить лояльность сотрудников,"\n\nЗадумываетесь, как увеличить благонадежнос..."
1,HR-кейсы,Excel для эйчаров: три кейса,"\n\n\t\t\t\t об авторе:\n\nНиколай Павлов, ..."
2,HR-кейсы,Может ли психолог быть эйчаром?,"\n\nОб авторе:\n\n\n\n\n\nАндрей Коновалов, ос..."
3,HR-кейсы,HR‑фишка #2. Как собирать рекомендации надёжнее,\n\r\n\t Вы когда-нибудь давали рекомендации н...
4,HR-кейсы,Как повысить отклик на вакансию,"\n\r\n\t Кандидат прислал интересное резюме, а..."
...,...,...,...
577,Управление персоналом,В кого и почему превращаются эйчары,"\n\n\n\n\r\n\t Гусеница становится куколкой, к..."
578,Управление персоналом,"Социальная политика компании: введение, термин...",\n\nОб авторе:\n\n\n\n\n\n\nНовикова Марина Ль...
579,Управление персоналом,Чем же управляет HR-менеджер?,\n\r\n\t Пришедшее из английского языка слово ...
580,Управление персоналом,Как оптимизировать работу подразделения (униве...,"\n\r\n\t В компании, численность которой превы..."


### Обработка текста методоми NLP 

In [168]:
stops = []
with open('stop-ru.txt', 'r', encoding='utf-8') as f:
    for word in f:
        stops.append(word.strip())
        print(word.strip())

а
аа
аба
абз
абсолютно
авторизоваться
активный
алло
алтухов
атмосфера
ах
б
беду
без
безусловно
белый
бесспорно
бизнеса
благодарение
благодарить
ближе
близ
близко
блин
бог
богом
богу
более
больше
большинство
большой
большую
браво
будем
будет
будете
будешь
будто
буду
будут
будучи
будь
будьте
бы
бывает
бывало
бывший
был
была
были
было
бытие
бытует
быть
в
в общем
важная
важно
важное
важные
важный
вам
вами
вас
ваш
ваша
ваше
вашего
вашему
ваши
вверх
ввиду
вдали
вдобавок
вдоль
вдруг
ведь
везде
век
веке
верите
верить
веришь
вернее
верно
вернуться
вероятнее
вероятно
вероятности
вес
вестимо
весь
весьма
вечер
вещи
вещь
взгляд
взыскательный
взятие
взять
вид
видать
видел
видеть
видимо
видимости
видит
видите
видишь
видно
видя
вишь
вкл
вкус
вместе
вместо
вне
вниз
внизу
внутренний
внутри
внутрь
во
во первых
во-первых
вовторых
вода
водится
возле
возможно
возможное
воистину
война
вокруг
воля
вон
вообрази
вообразите
вообще
вообще-то
вопрос
восемнадцатый
восемнадцать
восемь
восемьдесят
восьмой
вот
вперед


In [87]:
import nltk
from pymorphy2 import MorphAnalyzer
from nltk.corpus import stopwords

patterns = "[A-Za-z0-9!#$%&'()*+,./:;<=>?@[\]^_`{|}~—\"\-]+"
stopwords_ru = stopwords.words("russian")
morph = MorphAnalyzer()

def lemmatize(doc):
    doc = re.sub(patterns, ' ', doc)
    tokens = []
    for token in doc.split():
        if token and token not in stopwords_ru:
            token = token.strip()
            token = morph.normal_forms(token)[0]
            
            tokens.append(token)
    if len(tokens) > 2:
        return tokens
    return None

data_lem = []

for lem_text in article:
    data_lem.append(lemmatize(lem_text))

print(data_lem[0])
    

['задумываться', 'увеличить', 'благонадёжность', 'преданность', 'верность', 'сотрудник', 'ваш', 'организация', 'это', 'несложно', 'достаточно', 'разработать', 'некоторый', 'организационный', 'мероприятие', 'для', 'бизнес', 'лояльность', 'сотрудник', 'являться', 'полезный', 'явление', 'важно', 'сотрудник', 'готовый', 'работать', 'результат', 'понадобиться', 'идти', 'уступка', 'в', 'копенгагенский', 'школа', 'маркетинг', 'вычислить', 'лояльность', 'работник', 'увеличить', 'лояльность', 'клиент', 'подняться', 'при', 'лояльность', 'клиент', 'расти', 'прибыль', 'организация', 'следующий', 'квартал', 'увеличиться', 'согласно', 'исследование', 'половина', 'сотрудник', 'средний', 'компания', 'искать', 'другой', 'возможность', 'трудоустройство', 'и', 'неудивительный', 'многие', 'компания', 'ежегодно', 'терять', 'около', 'треть', 'свой', 'сотрудник', 'с', 'такой', 'цифра', 'лояльность', 'сотрудник', 'казаться', 'дело', 'прошлое', 'согласно', 'исследование', 'провести', 'центр', 'американский', '

In [86]:
df_data = {
    'Категория': category,
    'Название статьи': name_article,
    'Статья': article,
    'Лематизация текста': data_lem
}

df = pd.DataFrame(df_data)
df

,Категория,Название статьи,Статья,Лематизация текста
0,HR-кейсы,Как повысить лояльность сотрудников,"\n\nЗадумываетесь, как увеличить благонадежнос...","[задумываться, увеличить, благонадёжность, пре..."
1,HR-кейсы,Excel для эйчаров: три кейса,"\n\n\t\t\t\t об авторе:\n\nНиколай Павлов, ...","[автор, николай, павлов, профессиональный, тре..."
2,HR-кейсы,Может ли психолог быть эйчаром?,"\n\nОб авторе:\n\n\n\n\n\nАндрей Коновалов, ос...","[о, автор, андрей, коновалов, основатель, ‑, п..."
3,HR-кейсы,HR‑фишка #2. Как собирать рекомендации надёжнее,\n\r\n\t Вы когда-нибудь давали рекомендации н...,"[вы, давать, рекомендация, сотрудник, коллега,..."
4,HR-кейсы,Как повысить отклик на вакансию,"\n\r\n\t Кандидат прислал интересное резюме, а...","[кандидат, прислать, интересный, резюме, ваш, ..."
...,...,...,...,...
577,Управление персоналом,В кого и почему превращаются эйчары,"\n\n\n\n\r\n\t Гусеница становится куколкой, к...","[гусеница, становиться, куколка, куколка, бабо..."
578,Управление персоналом,"Социальная политика компании: введение, термин...",\n\nОб авторе:\n\n\n\n\n\n\nНовикова Марина Ль...,"[о, автор, новиков, марина, львович, бизнес, к..."
579,Управление персоналом,Чем же управляет HR-менеджер?,\n\r\n\t Пришедшее из английского языка слово ...,"[прийти, английский, язык, слово, «менеджер», ..."
580,Управление персоналом,Как оптимизировать работу подразделения (униве...,"\n\r\n\t В компании, численность которой превы...","[в, компания, численность, который, превышать,..."


In [91]:
from nltk import word_tokenize
from nltk.probability import FreqDist

text_one = data_lem[0]
fdist = FreqDist(text_one)
pprint(fdist)

{'«делегирование»': 1,
 '«заняты»': 1,
 '«лояльность': 1,
 '«обесцененными»': 1,
 '«переть': 1,
 '«помолвка»': 1,
 '«расширение': 1,
 'автор': 1,
 'адаптация': 1,
 'американский': 1,
 'атмосфера': 2,
 'безопасность': 1,
 'берг': 1,
 'бестселлер': 1,
 'бизнес': 1,
 'благонадёжность': 1,
 'боб': 1,
 'болезненный': 1,
 'больший': 1,
 'бонус': 1,
 'быстро': 1,
 'быть': 4,
 'в': 2,
 'важно': 2,
 'важный': 1,
 'ваш': 2,
 'верность': 1,
 'вероятность': 1,
 'вести': 1,
 'взаимный': 1,
 'вид': 1,
 'внимание': 2,
 'вносить': 1,
 'вовлечённость': 1,
 'возможностей»': 1,
 'возможность': 1,
 'вот': 1,
 'всепроникающий': 1,
 'всё': 1,
 'вы': 1,
 'выполнять': 4,
 'выслушивание': 1,
 'вычислить': 1,
 'вышеперечисленный': 1,
 'год': 2,
 'годовой': 1,
 'гордость': 1,
 'готовый': 1,
 'далее': 1,
 'дать': 2,
 'двадцать': 1,
 'действие': 1,
 'действительно': 1,
 'делать': 1,
 'дело': 4,
 'для': 2,
 'доверяют»': 1,
 'договариваться': 1,
 'должный': 3,
 'достаточно': 1,
 'друг': 5,
 'другой': 1,
 'ежегодно':

In [93]:
fdist.most_common(10)

[('сотрудник', 36),
 ('лояльность', 13),
 ('который', 11),
 ('свой', 9),
 ('компания', 8),
 ('это', 7),
 ('работа', 7),
 ('организация', 5),
 ('рабочий', 5),
 ('друг', 5)]

In [117]:
#Результат после Токенизации, Лематизации обратно в нормальный текст из списка в качестве разделителя пробел
article_result = []
for alem_article in data_lem:
    article_result.append(" ".join(alem_article))
print(article_result[0])

задумываться увеличить благонадёжность преданность верность сотрудник ваш организация это несложно достаточно разработать некоторый организационный мероприятие для бизнес лояльность сотрудник являться полезный явление важно сотрудник готовый работать результат понадобиться идти уступка в копенгагенский школа маркетинг вычислить лояльность работник увеличить лояльность клиент подняться при лояльность клиент расти прибыль организация следующий квартал увеличиться согласно исследование половина сотрудник средний компания искать другой возможность трудоустройство и неудивительный многие компания ежегодно терять около треть свой сотрудник с такой цифра лояльность сотрудник казаться дело прошлое согласно исследование провести центр американский прогресс помимо болезненный наём замена смена сотрудник стоить около средний зарплата сотрудник очевидный прививать лояльность свой работник стоить но это сделать вот несколько шаг который мочь предпринять сегодня создать повысить лояльность сотрудник

In [101]:
df_data = {
    'Категория': category,
    'Название статьи': name_article,
    'Статья': article,
    'Лематизация текста': data_lem,
    'Очищенный текст': article_result
}

df = pd.DataFrame(df_data)
df

,Категория,Название статьи,Статья,Лематизация текста,Очищенный текст
0,HR-кейсы,Как повысить лояльность сотрудников,"\n\nЗадумываетесь, как увеличить благонадежнос...","[задумываться, увеличить, благонадёжность, пре...",задумываться увеличить благонадёжность преданн...
1,HR-кейсы,Excel для эйчаров: три кейса,"\n\n\t\t\t\t об авторе:\n\nНиколай Павлов, ...","[автор, николай, павлов, профессиональный, тре...",автор николай павлов профессиональный тренер п...
2,HR-кейсы,Может ли психолог быть эйчаром?,"\n\nОб авторе:\n\n\n\n\n\nАндрей Коновалов, ос...","[о, автор, андрей, коновалов, основатель, ‑, п...",о автор андрей коновалов основатель ‑ психолог...
3,HR-кейсы,HR‑фишка #2. Как собирать рекомендации надёжнее,\n\r\n\t Вы когда-нибудь давали рекомендации н...,"[вы, давать, рекомендация, сотрудник, коллега,...",вы давать рекомендация сотрудник коллега если ...
4,HR-кейсы,Как повысить отклик на вакансию,"\n\r\n\t Кандидат прислал интересное резюме, а...","[кандидат, прислать, интересный, резюме, ваш, ...",кандидат прислать интересный резюме ваш ответн...
...,...,...,...,...,...
577,Управление персоналом,В кого и почему превращаются эйчары,"\n\n\n\n\r\n\t Гусеница становится куколкой, к...","[гусеница, становиться, куколка, куколка, бабо...",гусеница становиться куколка куколка бабочка г...
578,Управление персоналом,"Социальная политика компании: введение, термин...",\n\nОб авторе:\n\n\n\n\n\n\nНовикова Марина Ль...,"[о, автор, новиков, марина, львович, бизнес, к...",о автор новиков марина львович бизнес консульт...
579,Управление персоналом,Чем же управляет HR-менеджер?,\n\r\n\t Пришедшее из английского языка слово ...,"[прийти, английский, язык, слово, «менеджер», ...",прийти английский язык слово «менеджер» означа...
580,Управление персоналом,Как оптимизировать работу подразделения (униве...,"\n\r\n\t В компании, численность которой превы...","[в, компания, численность, который, превышать,...",в компания численность который превышать один ...


### Кластеризация

In [210]:
import pandas as pd
import numpy as np
import nltk
import re
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
cv = CountVectorizer(max_df=0.8, min_df=2, stop_words=stops)

count = cv.fit(article_result)
count = cv.transform(article_result)

df_vec = pd.DataFrame(count.toarray(), columns = cv.get_feature_names())
df_vec

,аббревиатура,абзац,абитуриент,абонемент,абонентский,абсолютный,абстрагироваться,абстрактный,абсурд,аванс,...,ясно,ясность,ясный,ячейка,ящик,ёлка,ёлочный,ёмкий,ёмко,ёмкость
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,1,0,0,0,0,...,0,0,0,6,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
577,0,0,0,0,0,0,0,0,0,0,...,1,0,1,0,0,0,0,0,0,0
578,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
579,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
580,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### Тематическое моделирование

In [208]:
from sklearn.decomposition import LatentDirichletAllocation
import random

LDA = LatentDirichletAllocation(n_components=5, random_state=42)
LDA.fit(count)

print('Топ 10 слов для наиболее подходяшей темы :)')
top_topic_words = first_topic.argsort()[-10:]
for i in top_topic_words:
    print(count_vect.get_feature_names()[i])

Топ 10 слов для наиболее подходяшей темы :)
текущий
опуститься
колбаса
отсидеть
приучать
результативный
предпенсионный
сей
приступать
неудержимый


In [209]:
j = 0
for i,topic in enumerate(LDA.components_):
    j += 1
    print(f'Топ 10 слов для тем статей #{j}:')
    print([count_vect.get_feature_names()[i] for i in topic.argsort()[-10:]])
    print('\n')

Топ 10 слов для тем статей #1:
['кровно', 'реквизит', 'лица', 'декорация', 'ой', 'превратиться', 'приурочить', 'зажигательный', 'отсеять', 'разделиться']


Топ 10 слов для тем статей #2:
['быстренько', 'первостепенный', 'отсеять', 'пункт', 'изготовление', 'если', 'разделиться', 'сервировка', 'оригинально', 'инспектор']


Топ 10 слов для тем статей #3:
['пункт', 'проведение', 'кола', 'бессрочный', 'текучки', 'лица', 'предотвращение', 'отсеять', 'результативность', 'неудачный']


Топ 10 слов для тем статей #4:
['оборот', 'спасатель', 'олень', 'скил', 'ссср', 'разместить', 'дешёвый', 'притом', 'стабилизироваться', 'приступ']


Топ 10 слов для тем статей #5:
['разделиться', 'лица', 'первое', 'исследуться', 'пункт', 'удлинённый', 'опустить', 'кам', 'кола', 'приступ']




In [200]:
reviews_datasets = []
topic_values = LDA.transform(count)
topic_values.shape
reviews_datasets = topic_values.argmax(axis=1) + 1

pprint(reviews_datasets)

array([1, 4, 4, 4, 5, 1, 4, 5, 4, 5, 5, 5, 1, 4, 3, 3, 5, 4, 4, 5, 4, 4,
       4, 5, 3, 3, 5, 1, 4, 5, 5, 3, 1, 5, 5, 3, 5, 5, 4, 2, 4, 5, 5, 1,
       1, 4, 3, 1, 1, 5, 1, 4, 5, 4, 4, 4, 4, 4, 4, 5, 1, 1, 4, 1, 4, 4,
       4, 4, 4, 3, 1, 3, 5, 5, 4, 3, 4, 4, 3, 1, 5, 1, 3, 4, 5, 4, 4, 1,
       4, 3, 5, 4, 4, 4, 4, 1, 4, 4, 4, 4, 4, 4, 4, 3, 1, 4, 1, 3, 4, 4,
       1, 1, 1, 5, 5, 4, 4, 4, 1, 1, 1, 3, 4, 4, 5, 4, 3, 4, 4, 4, 4, 4,
       1, 4, 3, 1, 4, 5, 1, 1, 1, 4, 5, 4, 4, 4, 1, 5, 3, 4, 3, 1, 3, 3,
       1, 3, 4, 1, 1, 1, 3, 4, 4, 1, 4, 4, 4, 4, 4, 5, 4, 1, 3, 1, 4, 3,
       3, 4, 1, 4, 4, 5, 5, 4, 3, 3, 4, 4, 5, 5, 5, 5, 1, 4, 4, 5, 1, 1,
       1, 4, 3, 5, 4, 4, 4, 1, 4, 4, 4, 4, 4, 5, 5, 4, 3, 4, 1, 3, 3, 5,
       4, 5, 4, 3, 1, 3, 5, 3, 1, 1, 4, 3, 4, 1, 4, 3, 3, 4, 3, 5, 4, 4,
       4, 5, 3, 4, 4, 3, 4, 3, 4, 3, 3, 4, 1, 5, 5, 3, 1, 4, 1, 5, 5, 5,
       4, 3, 4, 5, 4, 4, 1, 5, 4, 1, 5, 4, 4, 5, 5, 5, 1, 5, 4, 5, 4, 1,
       4, 5, 5, 5, 4, 5, 1, 1, 4, 1, 3, 5, 4, 4, 4,

### Присвоение темы статьи после тематического моделирования

In [201]:
df_data = {
    'Категория': category,
    'Название статьи': name_article,
    'Статья': article,
    'Лематизация текста': data_lem,
    'Очищенный текст': article_result,
    'Тема статьи': reviews_datasets
}

df = pd.DataFrame(df_data)
df

,Категория,Название статьи,Статья,Лематизация текста,Очищенный текст,Тема статьи
0,HR-кейсы,Как повысить лояльность сотрудников,"\n\nЗадумываетесь, как увеличить благонадежнос...","[задумываться, увеличить, благонадёжность, пре...",задумываться увеличить благонадёжность преданн...,1
1,HR-кейсы,Excel для эйчаров: три кейса,"\n\n\t\t\t\t об авторе:\n\nНиколай Павлов, ...","[автор, николай, павлов, профессиональный, тре...",автор николай павлов профессиональный тренер п...,4
2,HR-кейсы,Может ли психолог быть эйчаром?,"\n\nОб авторе:\n\n\n\n\n\nАндрей Коновалов, ос...","[о, автор, андрей, коновалов, основатель, ‑, п...",о автор андрей коновалов основатель ‑ психолог...,4
3,HR-кейсы,HR‑фишка #2. Как собирать рекомендации надёжнее,\n\r\n\t Вы когда-нибудь давали рекомендации н...,"[вы, давать, рекомендация, сотрудник, коллега,...",вы давать рекомендация сотрудник коллега если ...,4
4,HR-кейсы,Как повысить отклик на вакансию,"\n\r\n\t Кандидат прислал интересное резюме, а...","[кандидат, прислать, интересный, резюме, ваш, ...",кандидат прислать интересный резюме ваш ответн...,5
...,...,...,...,...,...,...
577,Управление персоналом,В кого и почему превращаются эйчары,"\n\n\n\n\r\n\t Гусеница становится куколкой, к...","[гусеница, становиться, куколка, куколка, бабо...",гусеница становиться куколка куколка бабочка г...,5
578,Управление персоналом,"Социальная политика компании: введение, термин...",\n\nОб авторе:\n\n\n\n\n\n\nНовикова Марина Ль...,"[о, автор, новиков, марина, львович, бизнес, к...",о автор новиков марина львович бизнес консульт...,1
579,Управление персоналом,Чем же управляет HR-менеджер?,\n\r\n\t Пришедшее из английского языка слово ...,"[прийти, английский, язык, слово, «менеджер», ...",прийти английский язык слово «менеджер» означа...,5
580,Управление персоналом,Как оптимизировать работу подразделения (униве...,"\n\r\n\t В компании, численность которой превы...","[в, компания, численность, который, превышать,...",в компания численность который превышать один ...,4
